<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF285 - Computación Científica </h1>
    <h2> Sylvester Equation with GMRes </h2>
    <h2> <a href="#acknowledgements"> [S]cientific [C]omputing [T]eam </a> </h2>
    <h2> Version: 1.03</h2>
</center>

<div id='toc' />

## Table of Contents
* [Sylvester Equation](#sylvester)
* [GMRes with afun(x)](#GMRes)
* [Using a Jacobi/Gauss-Seidel as iterative solver](#jacAndGSIterative)
* [Using the beautiful GMRes](#GMResTest)
* [Challenge](#chal)
* [Acknowledgements](#acknowledgements)

In [1]:
import numpy as np
import scipy as sp
from scipy import linalg as la
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['font.size'] = 14
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14

<div id='sylvester' />

# Sylvester Equation

The Sylvester Equation has the following form matricial form:
$$A\,X+X\,B=C$$
where $A\in\mathbb{R}^{n\times n}$, $B\in\mathbb{R}^{n\times n}$, $C\in\mathbb{R}^{n\times n}$ and $X\in\mathbb{R}^{n\times n}$.
$A$, $B$ and $C$ are given, the problem is to find the matrix $X$.
See https://en.wikipedia.org/wiki/Sylvester_equation.
Thus, an algorithm as PALU or classical version of iterative solvers can't be applied directly.
If you want to do so, please take a look to the ```vec``` operator: https://en.wikipedia.org/wiki/Vectorization_%28mathematics%29 , together with NumPy flatten(order='F') procedure and the NumPy Kronecker product of two arrays, np.kron.
Notice that if you really want to translate the Sylvester Equation into the tradicional form $\widehat{A}\,\widehat{\mathbf{x}}=\widehat{\mathbf{b}}$, just be aware that $\widehat{A}\in\mathbb{R}^{n^2\times n^2}$, $\widehat{\mathbf{x}}\in\mathbb{R}^{n^2}$, and $\widehat{\mathbf{b}}\in\mathbb{R}^{n^2}$.
This means that if $n$ is large, $n^2$ is huge!
And you may not have enough memory to store $\widehat{A}$.

## Vectorization: A quick review

In Mathematics the ```vec``` operator is an operator that translate a matrix into a vector as follows:
$$
\text{vec}
\left(
\begin{bmatrix}
  a & b \\
  c & d
\end{bmatrix}
\right)
=
\begin{bmatrix}
  a \\
  c \\
  b \\
  d
\end{bmatrix}.
$$
This can be achieved with the ```flatten``` function from NumPy with the parameter ```F```. Consider the following numerical example:

In [2]:
A = np.array([[1, 3],[2, 4]])
print(A)
print(A.flatten('F'))

[[1 3]
 [2 4]]
[1 2 3 4]


Why is the ```vec``` operator useful?
This operator is useful because of the following identity for the matrices $A$, $B$, and $C$ that belong to $\mathbb{R}^{n\times n}$:
$$
\text{vec}
\left(
A\,B\,C
\right)
=
(C^T \otimes A)\,\text{vec}(B),
$$
where $\otimes$ is the [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product).
This implies that the Sylvester equation $A\,X+X\,B=C$, after adding two identity matrices $I\in\mathbb{R}^{n\times n}$ conveniently we obtain,
$$
A\,X\,I+I\,X\,B=C,
$$
so, if we apply the ```vec``` operator we obtain,
$$
\begin{align}
    \text{vec}(A\,X\,I+I\,X\,B) & = \text{vec}(C)\\
    \text{vec}(A\,X\,I)+\text{vec}(I\,X\,B) & = \text{vec}(C)\\
    (I \otimes A)\,\text{vec}(X)+(B^T \otimes I)\text{vec}(X) & = \text{vec}(C)\\
    \left((I \otimes A)+(B^T \otimes I)\right)\text{vec}(X) & = \text{vec}(C).
\end{align}
$$
Thus, the true linear system we are solving is the following:
$$
\widehat{A}\,\widehat{\mathbf{x}}=\widehat{\mathbf{b}},
$$
where,
$$
\begin{align}
    \widehat{A} & = (I \otimes A)+(B^T \otimes I) \in \mathbb{R}^{n^2\times n^2},\\
    \widehat{\mathbf{x}} &= \text{vec}(X) \in \mathbb{R}^{n^2},\\
    \widehat{\mathbf{b}} &= \text{vec}(C) \in \mathbb{R}^{n^2}.
\end{align}
$$
Why don't we just use PALU with $\widehat{A}\,\widehat{\mathbf{x}}=\widehat{\mathbf{b}}$?
Because we may run out of memory!
Notice that the original matrices are of size $n \times n$, but $\widehat{A}$ is of size $n^2 \times n^2 $, which may be huge depending on the value of $n$!

<div id='GMRes' />

# GMRes with afun($\mathbf{x}$)
[Back to TOC](#toc)

GMRes is a member of the family of Krylov methods. It finds an approximation of $\mathbf{x}$ restricted to _live_ on the Krylov sub-space $\mathcal{K_k}$,  where $\mathcal{K_k}=\{\mathbf{r}_0, A\,\mathbf{r}_0, A^2\,\mathbf{r}_0, \cdots, A^{k-1}\,\mathbf{r}_0\}$ and $\mathbf{r}_0 = \mathbf{b} - A\,\mathbf{x}_0$ is the residual vector of the initial guess.

The idea behind this method is to look for improvements to the initial guess $\mathbf{x}_0$ in the Krylov space. At the $k$-th iteration, we enlarge the Krylov space by adding $A^k\,\mathbf{r}_0$, reorthogonalize the basis, and then use least squares to find the best improvement to add to $\mathbf{x}_0$.

The algorithm is as follows:

`Generalized Minimum Residual Method`

$\mathbf{x}_0$ `= initial guess`<br>
$\mathbf{r}$ `=` $\mathbf{b} - A\,\mathbf{x}_0$ `=` $\mathbf{b} - $<span style="color:blue">afun</span>$(\mathbf{x}_0)$<br>
$\mathbf{q}_1$ `=` $\mathbf{r} / \|\mathbf{r}\|_2$<br>
`for` $k = 1, ..., m$<br>
$\qquad \ \ \mathbf{y} = A\,\mathbf{q}_k$ = <span style="color:blue">afun</span>$(\mathbf{q}_k)$ <br>
$\qquad$ `for` $j = 1,2,...,k$ <br>
$\qquad \qquad$ $h_{jk} = \mathbf{q}_j^*\,\mathbf{y}$<br>
$\qquad \qquad$ $\mathbf{y} = \mathbf{y} - h_{jk}\, \mathbf{q}_j$<br>
$\qquad$ `end`<br>
$\qquad h_{k+1,k} = \|y\|_2 \qquad$ `(If ` $h_{k+1,k} = 0$ `, skip next line and terminate at bottom.)` <br>
$\qquad \ \mathbf{q}_{k+1} = \mathbf{y}/h_{k+1,k}$ <br>
$\qquad$ `Minimize` $\left\|\widehat{H}_k\, \mathbf{c}_k - [\|\mathbf{r}\|_2, \, 0, \, 0, \, ..., \, 0]^T \right\|_2$ `for` $\mathbf{c}_k$ <br>
$\qquad$ $\mathbf{x}_k = Q_k \, \mathbf{c}_k + \mathbf{x}_0$ <br>
`end`

<div id='jacAndGSIterative' />

# Using a Jacobi/Gauss-Seidel iterative solver
[Back to TOC](#toc)

The Sylvester Equation could be solve iteratively by building matrix-fix-point-iterative procedures.
For instance, we propose 2 algorihtms, the first one is derived as follows,
\begin{align*}
    A\,X+X\,B &= C,\\
    X &= A^{-1}\,(C-X\,B),\\
    X^{(0)} &= \underline{\underline{0}},\\
    X^{(i+1)} &= A^{-1}\,(C-X^{(i)}\,B),
\end{align*}
where $\underline{\underline{0}}$ is the matrix of dimension $n\times n$ o $0$.
Recall **please** that we don't compute the inverse of a matrix in general, i.e. we don't need to compute $A^{-1}$, but what we do is to solve the corresponding linear system of equations.
The second alternative is the following,
\begin{align*}
    A\,X+X\,B &= C,\\
    X &= (C-A\,X)\,B^{-1},\\
    X^{(0)} &= \underline{\underline{0}},\\
    X^{(i+1)} &= (C-A\,X^{(i)})\,B^{-1},
\end{align*}
where, again, **please** don't compute the inverse of $B$.
However in this case is a bit more challenging the implementation since it is a bit trickier to get the corresponding linear system of equation, just think out of the ```transpose```-box!

In [3]:
def solve_JGS_iterative_Sylvester(A,B,C,m,alg=1):
    if alg==1:
        # Algorithm 1
        # AX+XB=C
        # X=A^{-1}(C-XB)
        # X^{(i+1)}=A^{-1}(C-X^{(i)} B)
        X0 = np.zeros_like(A)
        X1 = np.zeros_like(A)
        for i in range(m):
            X1=np.linalg.solve(A,C-np.dot(X0,B))
            X0=X1
            # Just 'printing' a residual!
            print(np.linalg.norm(np.dot(A,X1)+np.dot(X1,B)-C))
        return X1
#    elif algo==2: # TO DO !!!!!!!!!!!!!
        # Algorithm 2
        # AX+XB=C
        # X = (C-AX)B^{-1}
        # X^{(i+1)}=(C-A X^{(i)})B^{-1}
        # How do we implement this? Hint: You only need to use np.linalg.solve in a convenient way.

In [4]:
# First TEST
n = 10
np.random.seed(0)
A = np.random.rand(n,n)+10*np.eye(n)
#print(np.linalg.eigvals(A))
B = np.random.rand(n,n)
#print(np.linalg.eigvals(B))
C = np.random.rand(n,n)

In [5]:
X_JGS=solve_JGS_iterative_Sylvester(A,B,C,10)

1.9907084066946081
0.8043165445235992
0.3633069939849517
0.18339713376573877
0.09990241167316857
0.05657422650777181
0.03258207341158652
0.018893884242102987
0.010986814278637508
0.00639621207872684


So, since the residual is decreasing we expect convergence. Let's look at the following example,

In [6]:
# Second TEST
n = 10
np.random.seed(0)
A = np.random.rand(n,n)+2*np.eye(n)
#print(np.linalg.eigvals(A))
B = np.random.rand(n,n)
#print(np.linalg.eigvals(B))
C = np.random.rand(n,n)

In [7]:
X_JGS=solve_JGS_iterative_Sylvester(A,B,C,10)

8.06674588003015
34.57406247675346
168.70353729762985
819.8919726957436
3949.6601997495295
18860.437124867964
89325.87084866346
419868.5695000343
1959858.3262074112
9089702.028396815


Unfortunately in this case it is divergent.
One alternativs is to implement Algorithm 2 and hope it will converge.
**A better alternative is to use GMRes!!**

<div id='GMResTest' />

# Using the beautiful GMRes
[Back to TOC](#toc)

In [8]:
# This function computes the 'matrix-vector' product of the matrix we don't have explicitly stored!!
def compute_matrix_vector_product(x,A,B,n):
    X = np.reshape(x,(n,n),order='F')
    out = np.dot(A,X)+np.dot(X,B)
    return out.flatten('F')
# This is part of the interface that SciPy requires.
Ax = lambda x: compute_matrix_vector_product(x,A,B,n)
# This is the famous 'afun'!!
afun = spla.LinearOperator((n**2, n**2), matvec=Ax)
# Just running GMRes
x, exitCode = spla.gmres(afun, C.flatten('F'), tol=1e-10)
# Just reshaping the 
X_GMRes = np.reshape(x,(n,n),order='F')
print('residual: ',np.linalg.norm(np.dot(A,X_GMRes)+np.dot(X_GMRes,B)-C))
#print(X_GMRes)

residual:  5.336422186956609e-10


/var/folders/9y/fwh0nfx16wd6199vf_40_xkr0000gn/T/ipykernel_45493/1318436619.py:11: DeprecationWarning: 'scipy.sparse.linalg.gmres' keyword argument `tol` is deprecated in favor of `rtol` and will be removed in SciPy v1.14.0. Until then, if set, it will override `rtol`.
  x, exitCode = spla.gmres(afun, C.flatten('F'), tol=1e-10)


This is beautiful!!
We were able to solve a linear system of equations without even requiring to build the 'large' matrix associated to the linear system!

## Computing the 'truly' relative residues

In [9]:
Ax_JGS = Ax(X_JGS.flatten(order='F'))
Ax_GMRes = Ax(X_GMRes.flatten(order='F'))
c =  C.flatten(order='F')
print(np.linalg.norm(Ax_JGS-c)/np.linalg.norm(c))
print(np.linalg.norm(Ax_GMRes-c)/np.linalg.norm(c))

1534827.066866048
9.010730150645104e-11


<div id='chal' />

# Challenge: What do we need to change in our implementation of GMRes to be able to use the lambda function "Ax"?
[Back to TOC](#toc)

In [10]:
# This is a very instructive implementation of GMRes.
def GMRes_Ax(A, b, x0=np.array([0.0]), m=10, flag_display=True, threshold=1e-12):
    n = len(b)
    if len(x0)==1:
        x0=np.zeros(n)
    r0 = b - np.dot(A, x0)
    nr0=np.linalg.norm(r0)
    out_res=np.array(nr0)
    Q = np.zeros((n,n))
    H = np.zeros((n,n))
    Q[:,0] = r0 / nr0
    flag_break=False
    for k in np.arange(np.min((m,n))):
        y = np.dot(A, Q[:,k])
        if flag_display:
            print('||y||=',np.linalg.norm(y))
        for j in np.arange(k+1):
            H[j][k] = np.dot(Q[:,j], y)
            if flag_display:
                print('H[',j,'][',k,']=',H[j][k])
            y = y - np.dot(H[j][k],Q[:,j])
            if flag_display:
                print('||y||=',np.linalg.norm(y))
        # All but the last equation are treated equally. Why?
        if k+1<n:
            H[k+1][k] = np.linalg.norm(y)
            if flag_display:
                print('H[',k+1,'][',k,']=',H[k+1][k])
            if (np.abs(H[k+1][k]) > 1e-16):
                Q[:,k+1] = y/H[k+1][k]
            else:
                print('flag_break has been activated')
                flag_break=True
            # Do you remember e_1? The canonical vector.
            e1 = np.zeros((k+1)+1)        
            e1[0]=1
            H_tilde=H[0:(k+1)+1,0:k+1]
        else:
            H_tilde=H[0:k+1,0:k+1]
        # Solving the 'SMALL' least square problem. 
        # This could be improved with Givens rotations!
        ck = np.linalg.lstsq(H_tilde, nr0*e1)[0] 
        if k+1<n:
            x = x0 + np.dot(Q[:,0:(k+1)], ck)
        else:
            x = x0 + np.dot(Q, ck)
        # Why is 'norm_small' equal to 'norm_full'?
        norm_small=np.linalg.norm(np.dot(H_tilde,ck)-nr0*e1)
        out_res = np.append(out_res,norm_small)
        if flag_display:
            norm_full=np.linalg.norm(b-np.dot(A,x))
            print('..........||b-A\,x_k||=',norm_full)
            print('..........||H_k\,c_k-nr0*e1||',norm_small);
        if flag_break:
            if flag_display: 
                print('EXIT: flag_break=True')
            break
        if norm_small<threshold:
            if flag_display:
                print('EXIT: norm_small<threshold')
            break
    return x,out_res

<>:52: SyntaxWarning: invalid escape sequence '\,'
<>:53: SyntaxWarning: invalid escape sequence '\,'
<>:52: SyntaxWarning: invalid escape sequence '\,'
<>:53: SyntaxWarning: invalid escape sequence '\,'
/var/folders/9y/fwh0nfx16wd6199vf_40_xkr0000gn/T/ipykernel_45493/1148402014.py:52: SyntaxWarning: invalid escape sequence '\,'
  print('..........||b-A\,x_k||=',norm_full)
/var/folders/9y/fwh0nfx16wd6199vf_40_xkr0000gn/T/ipykernel_45493/1148402014.py:53: SyntaxWarning: invalid escape sequence '\,'
  print('..........||H_k\,c_k-nr0*e1||',norm_small);


<div id='acknowledgements' />

# Acknowledgements
* _Material created by professor Claudio Torres_ (`ctorres@inf.utfsm.cl`). _July 2020._
* _Update June 2021 - v1.01 - C.Torres_ : Major update.
* _Update November 2021 - v1.02 - C.Torres_ : Adding ```vec``` explanation for building the associated large linear system of equations.
* _Update June 2022 - v1.03 - C.Torres_ : Fixing issue with backslash.